<a href="https://colab.research.google.com/github/tx871217/IndependentStudy_Mango_1/blob/master/Mango_SVM_Keys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# def

In [ ]:
import numpy as np
import cv2

def object_area(org_image):
  obj_area=0
  kernel = np.ones((5,5),np.uint8)
  gray = cv2.cvtColor(org_image, cv2.COLOR_BGR2GRAY)
  thresh = cv2.threshold(gray, 0 ,255, cv2.THRESH_OTSU)[1]
  thresh = cv2.erode(thresh,kernel,iterations=5)
  thresh = cv2.dilate(thresh,kernel,iterations=5)
  for i in range(thresh.shape[0]):
    for j in range(thresh.shape[1]):
      if gray[i][j]!=0:
        obj_area+=1#counting the whole object area

  return obj_area

def ostu_cc(org_image):
  #get gray
  gray = cv2.cvtColor(org_image, cv2.COLOR_BGR2GRAY)
  #get ostu
  kernel = np.ones((5,5),np.uint8)
  thresh = cv2.threshold(gray, 0 ,255, cv2.THRESH_OTSU)[1]
  thresh = cv2.erode(thresh,kernel,iterations=5)
  thresh = cv2.dilate(thresh,kernel,iterations=5)
  #get background
  bg = cv2.threshold(gray, 0 ,255, cv2.THRESH_BINARY)[1]
  kernel = np.ones((5,5),np.uint8)
  bg=cv2.erode(bg,kernel,iterations=7)
  #take off background
  obj_area=0
  defects_area=0
  new=[]
  for i in range(thresh.shape[0]):
    ar=[]
    for j in range(thresh.shape[1]):
      if bg[i][j]==0:# if this pixel is black
        ar.append(255)# make the same position in new image is white
      else:
        ar.append(thresh[i][j])# or append the original pixel
      if ar[j]==0:
        defects_area+=1#counting the black pixels
      if gray[i][j]!=0:
        obj_area+=1#counting the whole object area
    new.append(ar)
  area_rate=defects_area/obj_area
  new=np.array(new,dtype=np.uint8) # the numpy default dtype is int64, it can't be used in cvtColor
  #reverse b/w
  new_convert=[]
  for i in range(new.shape[0]):
    array=[]
    for j in range(new.shape[1]):
      if new[i][j] == 255:
        array.append(0)
      else:
        array.append(255)
    new_convert.append(array)
  new_convert=np.array(new_convert,dtype=np.uint8)
  #convert defects with colors
  num,labels=cv2.connectedComponents(new_convert)
  if (num-1)==0:
    average_rate=0
  else:
    average_rate=area_rate/(num-1)
  return [area_rate,num-1,average_rate]

from skimage.feature import hog
def hog_rates(hog_image,obj_size):# not using
  counts=[]
  for i in range(0,255):
    counts.append(0)
  for i in hog_image:
    for j in i:
      if j!=0:
        counts[int(j)]+=1
  for i in range(len(counts)):
    counts[i]=counts[i]/obj_size
  return counts

def RGB_rates(image):
  rates=[]
  b, g, r = cv2.split(image)
  for i in [r,g,b]:
    rgb_count=0
    for j in i:
      for k in j:
        rgb_count+=k
    rates.append(rgb_count)
  return [rates[0]/sum(rates),rates[1]/sum(rates),rates[2]/sum(rates)]

def uneven(image):
  crop_base=4
  crop_img=[]
  all_rgb=[]
  r_all=0
  g_all=0
  b_all=0
  for i in range(crop_base):
    for j in range(crop_base):
      crop_img.append(resized[int(resized.shape[0]*j/crop_base):int(resized.shape[0]*(1+j)/crop_base), int(resized.shape[1]*i/crop_base):int(resized.shape[1]*(1+i)/crop_base)])
      all_rgb.append(RGB_rates(crop_img[j+crop_base*i]))
      r_all+=RGB_rates(crop_img[j+crop_base*i])[0]
      g_all+=RGB_rates(crop_img[j+crop_base*i])[1]
      b_all+=RGB_rates(crop_img[j+crop_base*i])[2]
  r_average=r_all/crop_base**2
  g_average=g_all/crop_base**2
  b_average=b_all/crop_base**2
  r_deviation=0
  g_deviation=0
  b_deviation=0
  for i in all_rgb:
    r_deviation+=abs(r_average-i[0])
    g_deviation+=abs(g_average-i[1])
    b_deviation+=abs(b_average-i[2])
  r_average_deviation=r_deviation/crop_base**2
  g_average_deviation=g_deviation/crop_base**2
  b_average_deviation=b_deviation/crop_base**2
  return [r_average_deviation,g_average_deviation,b_average_deviation]

def list_features():
  features=["id,","defects_rates,","defects_sum,","average_rates,","R_rates,","G_rates,","B_rates,","hog_mango,","hog_unit,","hog_mango_trans,","hog_unit_trans,","r_dv,","g_dv,","b_dv,","target"]
  return features

# run

In [ ]:
import os
from tqdm import tqdm
progress = tqdm(total=6400)

folders=[]
folder_path = '/content/drive/My Drive/img/'
folders.append(folder_path+"Train/A")
folders.append(folder_path+"Train/B")
folders.append(folder_path+"Train/C")
folders.append(folder_path+"Dev/A")
folders.append(folder_path+"Dev/B")
folders.append(folder_path+"Dev/C")

folder_content = os.listdir(folder_path)
LF=list_features()
csv_file = open("/content/drive/My Drive/data/mango_keys6.csv","w")

# write features(first row)
for i in LF:
  csv_file.write(i)
csv_file.write("\n")
# wirte contents
for c in folders:
  folder_content = os.listdir(c)
  for item in folder_content:
    progress.update(1)
    f = c +"/"+ item
    org_image = cv2.imread(f)
    resized = cv2.resize(org_image, (org_image.shape[1]//5, org_image.shape[0]//5), interpolation = cv2.INTER_AREA)
    csv_file.write(item+",")# write id
    obj_size=object_area(resized)
    for i in ostu_cc(resized):
      csv_file.write(str(i)+",")
    rgb_list=RGB_rates(resized)
    for i in rgb_list:
      csv_file.write(str(i)+",")
    fd = hog(resized, orientations=9, pixels_per_cell=(8, 8),
                      cells_per_block=(2, 2),transform_sqrt=False, visualize=False, multichannel=True)
    csv_file.write(str(sum(fd)/obj_size)+",")
    csv_file.write(str(sum(fd)/len(fd))+",")
    fd = hog(resized, orientations=9, pixels_per_cell=(8, 8),
                      cells_per_block=(2, 2),transform_sqrt=True, visualize=False, multichannel=True)
    csv_file.write(str(sum(fd)/obj_size)+",")
    csv_file.write(str(sum(fd)/len(fd))+",")
    for i in uneven(resized):
      csv_file.write(str(i)+",")
    csv_file.write(c[-1])
    csv_file.write("\n")
csv_file.close()

# train

In [ ]:
import pandas
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

mango = pandas.read_csv("/content/drive/My Drive/data/mango_keys5.csv") 
X = mango.drop("target",axis=1).drop("id",axis=1).values
# X = mango.drop("target",axis=1).drop("id",axis=1).drop("R_rates",axis=1).drop("G_rates",axis=1).drop("B_rates",axis=1).values
Y = mango["target"].values
label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(Y)
X = StandardScaler().fit_transform(X)
X_train , X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2)
clf = SVC(kernel='rbf').fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[302  69  30]
 [122 277  82]
 [ 69 103 226]]
              precision    recall  f1-score   support

           0       0.61      0.75      0.68       401
           1       0.62      0.58      0.60       481
           2       0.67      0.57      0.61       398

    accuracy                           0.63      1280
   macro avg       0.63      0.63      0.63      1280
weighted avg       0.63      0.63      0.63      1280



In [ ]:
mango.head()

,id,defects_rates,defects_sum,average_rates,R_rates,G_rates,B_rates,hog_mango,hog_unit,hog_mango_trans,hog_unit_trans,r_dv,g_dv,b_dv,target
0,03677.jpg,0.036236,3,0.012079,0.473229,0.266067,0.260705,0.085942,0.124441,0.085445,0.123721,0.015334,0.016651,0.015760,A
1,03678.jpg,0.000000,0,0.000000,0.487624,0.285370,0.227006,0.085425,0.121383,0.085246,0.121130,0.023388,0.011978,0.020161,A
2,03681.jpg,0.062080,2,0.031040,0.499424,0.231137,0.269439,0.082917,0.128099,0.082767,0.127866,0.038514,0.015811,0.022703,A
3,03691.jpg,0.089103,1,0.089103,0.557514,0.238038,0.204448,0.080806,0.130571,0.080981,0.130852,0.053681,0.028472,0.030642,A
4,03685.jpg,0.000000,0,0.000000,0.530571,0.267023,0.202406,0.085103,0.123345,0.085069,0.123296,0.050539,0.024565,0.030549,A
